#  **时间序列预测——商店销售额**
- 使用的特征包含时间序列的基本特征和其他相关特征，尝试使用销售额滚动特征，如过去7日滚动平均销售额和标准差，但表现不尽人意
- 模型使用未调参的随机森林和LightGBM，在验证集上的表现良好，两个验证集的平均均方根对数误差为0.23
- 将多次预测结果提交至Kaggle，平均得分为0.54，比最佳得分0.37只高出17%的偏差

In [1]:
import numpy as np
import pandas as pd
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_log_error
from statsmodels.tsa.deterministic import DeterministicProcess

## **一、数据加载**

In [2]:
file_path = './data/'

In [3]:
train = pd.read_csv(
    file_path + 'train.csv',
    parse_dates=['date'],
    dtype={
        'store_nbr': 'category',
        'family': 'category',
        'sales': 'float32',
        'onpromotion': 'uint64'
    }
)
train.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [4]:
test = pd.read_csv(
    file_path + 'test.csv',
    parse_dates=['date'],
    dtype={
        'store_nbr': 'category',
        'family': 'category',
        'sales': 'float32',
        'onpromotion': 'uint64'
    }
)
test.head()

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0


In [5]:
stores = pd.read_csv(
    file_path + 'stores.csv',
     dtype={'store_nbr': 'category', 'cluster': 'uint64'},
     usecols=['store_nbr', 'cluster']
)
stores.head()

,store_nbr,cluster
0,1,13
1,2,13
2,3,8
3,4,9
4,5,4


In [6]:
holidays = pd.read_csv(
   file_path + 'holidays_events.csv',
    parse_dates=['date'],
    dtype={'locale': 'str'},
    usecols=['date', 'locale']
)
holidays.head()

,date,locale
0,2012-03-02,Local
1,2012-04-01,Regional
2,2012-04-12,Local
3,2012-04-14,Local
4,2012-04-21,Local


In [7]:
oil = pd.read_csv(file_path + 'oil.csv', parse_dates=['date'], dtype={ 'dcoilwtico': 'float32'})
oil.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.139999
2,2013-01-03,92.970001
3,2013-01-04,93.120003
4,2013-01-07,93.199997


## **二、数据清洗**

### **1、空值检测**

In [8]:
series_lis = [
    train.isnull().sum(),
    test.isnull().sum(),
    stores.isnull().sum(),
    holidays.isnull().sum(),
    oil.isnull().sum()
]
series_names = ['train', 'test', 'stores', 'holidays', 'oil']

In [9]:
for s, s_name in zip(series_lis, series_names):
    if s.sum() != 0:
        print(f'{s_name}存在空值：')
        print(s)

oil存在空值：
date           0
dcoilwtico    43
dtype: int64


### **2、空值填充**

In [10]:
oil.bfill(inplace=True)
oil.isnull().sum()

date          0
dcoilwtico    0
dtype: int64

## **三、特征工程**

### **1、训练集**

In [11]:
train_data = train.copy()
train_data.shape

(3000888, 6)

In [12]:
# 趋势特征
dp = DeterministicProcess(
    index=train_data.set_index(['date', 'store_nbr', 'family']).unstack(['store_nbr', 'family']).index,
    order=1
)
trend = dp.in_sample().reset_index()
train_data = pd.merge(left=train_data, right=trend, on='date', how='left')
# 商店类别特征
train_data = pd.merge(left=train_data, right=stores, on='store_nbr', how='left')
train_data['store_nbr'] = LabelEncoder().fit_transform(y=train_data.store_nbr)
# 油价特征
train_data = pd.merge(left=train_data, right=oil, on='date', how='left').ffill()
# 商品类别特别编码
train_data['family'] = LabelEncoder().fit_transform(y=train_data.family)
# 发薪日特征
condition = ((train_data.date.dt.day == 15) | (train_data.date == train_data.date+pd.offsets.MonthEnd(0)))
train_data['payday'] = 0
train_data.loc[condition, 'payday'] = 1
# 突发重大自然灾害特征
train_data['disaster'] = 0
train_data.loc[train_data.date == '2016-04-16', 'disaster'] = 1
# 年的日特征
train_data = train_data.set_index('date').to_period('D')
train_data['dayofyear'] = train_data.index.dayofyear
# 月的日特征
train_data['day'] = train_data.index.day
# 月特征
train_data['month'] = train_data.index.month
# 周的日特征
train_data['dayofweek'] = train_data.index.dayofweek
# 节假日特征
train_data = pd.merge(left=train_data, right=holidays, on='date', how='left')
train_data.locale = train_data.locale.fillna(value='ordinary day')
train_data['locale'] = LabelEncoder().fit_transform(y=train_data.locale)
train_data.set_index('date', inplace=True)
print(train_data.shape)
train_data.head()

(3000888, 15)


,id,store_nbr,family,sales,onpromotion,trend,cluster,dcoilwtico,payday,disaster,dayofyear,day,month,dayofweek,locale
date,,,,,,,,,,,,,,,
2013-01-01,0,0,0,0.0,0,1.0,13,93.139999,0,0,1,1,1,1,0
2013-01-01,1,0,1,0.0,0,1.0,13,93.139999,0,0,1,1,1,1,0
2013-01-01,2,0,2,0.0,0,1.0,13,93.139999,0,0,1,1,1,1,0
2013-01-01,3,0,3,0.0,0,1.0,13,93.139999,0,0,1,1,1,1,0
2013-01-01,4,0,4,0.0,0,1.0,13,93.139999,0,0,1,1,1,1,0


### **2、测试集**

In [13]:
test_data = test.copy()
test_data.shape

(28512, 5)

In [14]:
trend_test = dp.out_of_sample(
    forecast_index=test_data.set_index(['date', 'store_nbr', 'family']).unstack(['store_nbr', 'family']).index,
    steps=16
).reset_index()
test_data = pd.merge(left=test_data, right=trend_test, on='date', how='left')
test_data = pd.merge(left=test_data, right=stores, on='store_nbr', how='left')
test_data = pd.merge(left=test_data, right=oil, on='date', how='left').ffill()
test_data['store_nbr'] = LabelEncoder().fit_transform(y=test_data.store_nbr)
test_data['family'] = LabelEncoder().fit_transform(y=test_data.family)
condition = ((test_data.date.dt.day == 15) | (test_data.date == test_data.date+pd.offsets.MonthEnd(0)))
test_data['payday'] = 0
test_data.loc[condition, 'payday'] = 1
test_data['disaster'] = 0
test_data = test_data.set_index('date').to_period('D')
test_data['dayofyear'] = test_data.index.dayofyear
test_data['day'] = test_data.index.day
test_data['month'] = test_data.index.month
test_data['dayofweek'] = test_data.index.dayofweek
test_data = pd.merge(left=test_data, right=holidays, on='date', how='left')
test_data.locale = test_data.locale.fillna(value='Local')
test_data['locale'] = LabelEncoder().fit_transform(y=test_data.locale)
test_data.set_index('date', inplace=True)
print(test_data.shape)
test_data.head()

(28512, 14)


,id,store_nbr,family,onpromotion,trend,cluster,dcoilwtico,payday,disaster,dayofyear,day,month,dayofweek,locale
date,,,,,,,,,,,,,,
2017-08-16,3000888,0,0,0,1685.0,13,46.799999,0,0,228,16,8,2,0
2017-08-16,3000889,0,1,0,1685.0,13,46.799999,0,0,228,16,8,2,0
2017-08-16,3000890,0,2,2,1685.0,13,46.799999,0,0,228,16,8,2,0
2017-08-16,3000891,0,3,20,1685.0,13,46.799999,0,0,228,16,8,2,0
2017-08-16,3000892,0,4,0,1685.0,13,46.799999,0,0,228,16,8,2,0


## **四、模型训练与评估**

### **1、划分数据集**

In [15]:
X, y = train_data.drop(columns=['id', 'sales']), train_data['sales']
X_train, y_train = X.loc['2013':'2017-07-15'], y.loc['2013':'2017-07-15']
display(X.tail(), X_train.tail())

,store_nbr,family,onpromotion,trend,cluster,dcoilwtico,payday,disaster,dayofyear,day,month,dayofweek,locale
date,,,,,,,,,,,,,
2017-08-15,53,28,0,1684.0,6,47.57,1,0,227,15,8,1,0
2017-08-15,53,29,1,1684.0,6,47.57,1,0,227,15,8,1,0
2017-08-15,53,30,148,1684.0,6,47.57,1,0,227,15,8,1,0
2017-08-15,53,31,8,1684.0,6,47.57,1,0,227,15,8,1,0
2017-08-15,53,32,0,1684.0,6,47.57,1,0,227,15,8,1,0


,store_nbr,family,onpromotion,trend,cluster,dcoilwtico,payday,disaster,dayofyear,day,month,dayofweek,locale
date,,,,,,,,,,,,,
2017-07-15,53,28,0,1653.0,6,46.529999,1,0,196,15,7,5,0
2017-07-15,53,29,0,1653.0,6,46.529999,1,0,196,15,7,5,0
2017-07-15,53,30,16,1653.0,6,46.529999,1,0,196,15,7,5,0
2017-07-15,53,31,0,1653.0,6,46.529999,1,0,196,15,7,5,0
2017-07-15,53,32,4,1653.0,6,46.529999,1,0,196,15,7,5,0


In [16]:
# 验证集1：2017-07-16开始的未来16天
X_val_1, y_val_1 = X.loc['2017-07-16':'2017-07-31'], y.loc['2017-07-16':'2017-07-31']
# 验证集2：2017-07-16开始的未来31天
X_val_2, y_val_2 = X.loc['2017-07-16':], y.loc['2017-07-16':]
display(X_val_1.tail(), X_val_2.tail())

,store_nbr,family,onpromotion,trend,cluster,dcoilwtico,payday,disaster,dayofyear,day,month,dayofweek,locale
date,,,,,,,,,,,,,
2017-07-31,53,28,1,1669.0,6,50.209999,1,0,212,31,7,0,0
2017-07-31,53,29,0,1669.0,6,50.209999,1,0,212,31,7,0,0
2017-07-31,53,30,6,1669.0,6,50.209999,1,0,212,31,7,0,0
2017-07-31,53,31,6,1669.0,6,50.209999,1,0,212,31,7,0,0
2017-07-31,53,32,0,1669.0,6,50.209999,1,0,212,31,7,0,0


,store_nbr,family,onpromotion,trend,cluster,dcoilwtico,payday,disaster,dayofyear,day,month,dayofweek,locale
date,,,,,,,,,,,,,
2017-08-15,53,28,0,1684.0,6,47.57,1,0,227,15,8,1,0
2017-08-15,53,29,1,1684.0,6,47.57,1,0,227,15,8,1,0
2017-08-15,53,30,148,1684.0,6,47.57,1,0,227,15,8,1,0
2017-08-15,53,31,8,1684.0,6,47.57,1,0,227,15,8,1,0
2017-08-15,53,32,0,1684.0,6,47.57,1,0,227,15,8,1,0


### **2、训练模型**

In [17]:
model_1 = RandomForestRegressor(n_jobs=-1)
model_2 = LGBMRegressor()

# 随机森林：学习数据的全局趋势和长期季节性
model_1.fit(X_train, y_train)
y_fit = model_1.predict(X_train)

# LightGBM: 学习局部非线性关系和短期残差波动
model_2.fit(X_train, y_train-y_fit)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.060461 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1126
[LightGBM] [Info] Number of data points in the train set: 2945646, number of used features: 12
[LightGBM] [Info] Start training from score -0.354938


,boosting_type,'gbdt'
,num_leaves,31
,max_depth,-1
,learning_rate,0.1
,n_estimators,100
,subsample_for_bin,200000
,objective,None
,class_weight,None
,min_split_gain,0.0
,min_child_weight,0.001
,min_child_samples,20


### **3、评估模型预测效果**

In [18]:
y_pred_1 = (model_1.predict(X_val_1) + model_2.predict(X_val_1)).clip(0.0)
y_pred_2 = (model_1.predict(X_val_2) + model_2.predict(X_val_2)).clip(0.0)
RMSLE_1 = mean_squared_log_error(y_val_1, y_pred_1)
RMSLE_2 = mean_squared_log_error(y_val_2, y_pred_2)
print(f'验证集1的均方根对误差为：{RMSLE_1:.3f}')
print(f'验证集2的均方根对误差为：{RMSLE_2:.3f}')
print('平均均方根对数误差为：  ', round(np.mean([RMSLE_1, RMSLE_2]), 3))

验证集1的均方根对误差为：0.222
验证集2的均方根对误差为：0.232
平均均方根对数误差为：   0.227


### **4、训练最终模型**

In [19]:
model_1 = RandomForestRegressor(n_jobs=-1)
model_2 = LGBMRegressor()

model_1.fit(X, y)
y_fit_full = model_1.predict(X)

model_2.fit(X, y-y_fit_full)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.059487 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1126
[LightGBM] [Info] Number of data points in the train set: 3000888, number of used features: 12
[LightGBM] [Info] Start training from score -0.415822


,boosting_type,'gbdt'
,num_leaves,31
,max_depth,-1
,learning_rate,0.1
,n_estimators,100
,subsample_for_bin,200000
,objective,None
,class_weight,None
,min_split_gain,0.0
,min_child_weight,0.001
,min_child_samples,20


## **五、预测**

In [20]:
X_test = test_data.drop(columns=['id'])
X_test.head()

,store_nbr,family,onpromotion,trend,cluster,dcoilwtico,payday,disaster,dayofyear,day,month,dayofweek,locale
date,,,,,,,,,,,,,
2017-08-16,0,0,0,1685.0,13,46.799999,0,0,228,16,8,2,0
2017-08-16,0,1,0,1685.0,13,46.799999,0,0,228,16,8,2,0
2017-08-16,0,2,2,1685.0,13,46.799999,0,0,228,16,8,2,0
2017-08-16,0,3,20,1685.0,13,46.799999,0,0,228,16,8,2,0
2017-08-16,0,4,0,1685.0,13,46.799999,0,0,228,16,8,2,0


In [21]:
# 预测2017-08-16开始未来16天的销售数据
y_pred = pd.Series((model_1.predict(X_test) + model_2.predict(X_test)).clip(0.0), index=test_data.id, name='sales')
y_pred.to_csv(file_path + 'submission.csv')